In [ ]:
sc.setLogLevel("ERROR")

In [ ]:
! cp  -rf /usr/lib/hive/conf/hive-site.xml   /usr/lib/spark/conf/

# 1. Data Understanding

## Collecting the current data from the LendingClub's Website.

In [ ]:
! rm -rf LoanStats_web.csv

In [ ]:
! touch LoanStats_web.csv

In [ ]:
! ls -l LoanStats*

In [ ]:
! rm -rf LoanStats*

In [ ]:
! wget https://resources.lendingclub.com/LoanStats_2016Q1.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2016Q2.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2016Q3.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2016Q4.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2017Q1.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2017Q2.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2017Q3.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2017Q4.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2018Q1.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2018Q2.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2018Q3.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
! wget https://resources.lendingclub.com/LoanStats_2019Q1.csv.zip
    
    

In [ ]:
! unzip LoanStats_2016Q1.csv.zip
! unzip LoanStats_2016Q2.csv.zip
! unzip LoanStats_2016Q3.csv.zip
! unzip LoanStats_2016Q4.csv.zip
! unzip LoanStats_2017Q1.csv.zip
! unzip LoanStats_2017Q2.csv.zip
! unzip LoanStats_2017Q3.csv.zip
! unzip LoanStats_2017Q4.csv.zip
! unzip LoanStats_2018Q1.csv.zip
! unzip LoanStats_2018Q2.csv.zip
! unzip LoanStats_2018Q3.csv.zip
! unzip LoanStats_2018Q4.csv.zip
! unzip LoanStats_2019Q1.csv.zip

In [ ]:
! sed -n '2,$p' LoanStats_2016Q1.csv > LoanStats_2016Q1-1.csv
! sed -n '3,$p' LoanStats_2016Q2.csv > LoanStats_2016Q2-1.csv
! sed -n '3,$p' LoanStats_2016Q3.csv > LoanStats_2016Q3-1.csv
! sed -n '3,$p' LoanStats_2016Q4.csv > LoanStats_2016Q4-1.csv
! sed -n '3,$p' LoanStats_2017Q1.csv > LoanStats_2017Q1-1.csv
! sed -n '3,$p' LoanStats_2017Q1.csv > LoanStats_2017Q2-1.csv
! sed -n '3,$p' LoanStats_2017Q1.csv > LoanStats_2017Q3-1.csv
! sed -n '3,$p' LoanStats_2017Q1.csv > LoanStats_2017Q4-1.csv
! sed -n '3,$p' LoanStats_2018Q1.csv > LoanStats_2018Q1-1.csv
! sed -n '3,$p' LoanStats_2018Q2.csv > LoanStats_2018Q2-1.csv
! sed -n '3,$p' LoanStats_2018Q3.csv > LoanStats_2018Q3-1.csv
! sed -n '3,$p' LoanStats_2018Q4.csv > LoanStats_2018Q4-1.csv
! sed -n '3,$p' LoanStats_2019Q1.csv > LoanStats_2019Q1-1.csv

In [ ]:
cat LoanStats_2016Q1-1.csv LoanStats_2016Q2-1.csv LoanStats_2016Q3-1.csv LoanStats_2016Q4-1.csv LoanStats_2017Q1-1.csv LoanStats_2017Q2-1.csv LoanStats_2017Q3-1.csv LoanStats_2017Q4-1.csv LoanStats_2018Q1-1.csv LoanStats_2018Q2-1.csv LoanStats_2018Q3-1.csv LoanStats_2018Q4-1.csv LoanStats_2019Q1-1.csv > LoanStats_web.csv

In [ ]:
! ls -lh LoanStats_web.csv

In [ ]:
! head -35 LoanStats_web.csv

In [ ]:
! wc -l LoanStats_web.csv

In [ ]:
! hdfs dfs -rm /user/root/LoanStats_web.csv

In [ ]:
! hdfs dfs -put LoanStats_web.csv /user/root/

In [1]:
! hdfs dfs -ls /user/root/

Found 3 items
drwxr-xr-x   - root hadoop          0 2019-10-11 15:34 /user/root/.sparkStaging
-rw-r--r--   2 root hadoop 1160195236 2019-10-11 13:23 /user/root/LoanStats_web.csv
drwxr-xr-x   - root hadoop          0 2019-10-11 13:27 /user/root/datacruncher


In [2]:
raw_df = spark.read.format('csv').option('header','true').option('mode','DROPMALFORMED')\
.load('/user/root/LoanStats_web.csv')

In [3]:
raw201617_df = raw_df

In [4]:
raw201617_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

### Numbers of Fields [1]

In [5]:
len(raw201617_df.columns)

144

In [6]:
rawweb_df = raw201617_df.select('id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 #'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m')

### Numbers of Fields after attributes selections [1]

In [7]:
len(rawweb_df.columns)

73

In [8]:
rawweb_df.select(['issue_d']).distinct().show()

+--------+
| issue_d|
+--------+
|Oct-2016|
|Feb-2019|
|Mar-2018|
|Sep-2018|
|Jan-2016|
|Jul-2016|
|Aug-2016|
|Feb-2017|
|Nov-2018|
|Jul-2018|
|Apr-2018|
|Dec-2018|
|May-2016|
|Jan-2018|
|Aug-2018|
|Jan-2019|
|Apr-2016|
|Sep-2016|
|Oct-2018|
|Feb-2018|
+--------+
only showing top 20 rows



In [9]:
raw_df = rawweb_df

In [10]:
raw_df.count()

1432439

In [11]:
df_colfam5 = raw_df.select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d")

### Numbers of Fields after final attributes selections [1]

In [12]:
len(df_colfam5.columns)

22

In [13]:
! pip install pandas

In [14]:
import pandas as pd
pd.DataFrame(df_colfam5.describe().take(6), columns=df_colfam5.describe().columns).transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,1432439,15370.392945179516,9646.028077240388,1000,9975
term,1432439,None,None,36 months,60 months
int_rate,1432439,None,None,5.31%,30.99%
installment,1432439,456.68848106617884,281.71353779219436,100.01,999.98
grade,1432439,None,None,A,G
emp_length,1432439,None,None,1 year,n/a
home_ownership,1432439,None,None,ANY,RENT
annual_inc,1432439,81034.59412687029,134183.4031357177,0,99999.84
verification_status,1432439,None,None,Not Verified,Verified


In [15]:
df_colfam5.select(['last_credit_pull_d']).distinct().\
orderBy(['last_credit_pull_d'],ascending=1).show()

+------------------+
|last_credit_pull_d|
+------------------+
|              null|
|          Apr-2016|
|          Apr-2017|
|          Apr-2018|
|          Apr-2019|
|          Aug-2016|
|          Aug-2017|
|          Aug-2018|
|          Aug-2019|
|          Dec-2015|
|          Dec-2016|
|          Dec-2017|
|          Dec-2018|
|          Feb-2016|
|          Feb-2017|
|          Feb-2018|
|          Feb-2019|
|          Jan-2016|
|          Jan-2017|
|          Jan-2018|
+------------------+
only showing top 20 rows



In [16]:
df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5.last_credit_pull_d == '').count()

0

In [17]:
df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5['last_credit_pull_d'].isNull()).count()

24

# 2. Data Preparation

Divide this process into 2 parts. The first is a business oriented preparation that turn many business rules to be programming's logics. Its result benefits many tasks related to the Business Intelligence and other descriptive analytics. The second one is a data science oriented preparation that turn many requirements of data science to be programming's logics. Its result benefits many tasks related to the Predictive analytics.

Asumption: Business oriented preparation: No missing values, ONLY month, Correct data types.

## Data Cleansing: Remove missing values

In [18]:
df_no_missing = df_colfam5.dropna(how='any')

## Check for a schema

In [19]:
df_no_missing.printSchema()

root
 |-- loan_amnt: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [20]:
df_no_missing_fitmem = df_no_missing.repartition(60)

In [21]:
df_no_missing_cached = df_no_missing_fitmem.cache()

In [22]:
df_no_missing_cached.registerTempTable("df")

## Number of data rows

In [23]:
df_no_missing_cached.count()

1429645

## Data Transformation: Remove Sign of Percent and Extract Month.

In [24]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove Sign of Percent

In [25]:
def f_removepercentsign(origin):
    return origin.rstrip('%')

In [26]:
removepercentsign = udf(lambda x: f_removepercentsign(x),StringType())

### Extract Month

In [27]:
def f_extractmonth(origin):
    return origin.split('-')[0]

In [28]:
extractmonth = udf(lambda x: f_extractmonth(x),StringType())

In [29]:
#df_no_missing_cached.withColumn('dti',df_no_missing_cached['dti'].cast(DoubleType())).show()

In [30]:
dfWithCrunch = df_no_missing_cached.\
withColumn('revol_util',removepercentsign(df_no_missing_cached['revol_util']).cast(DoubleType())).\
withColumn('int_rate',removepercentsign(df_no_missing_cached['int_rate']).cast(DoubleType())).\
withColumn('earliest_cr_line',extractmonth(df_no_missing_cached['earliest_cr_line']).cast(StringType())).\
withColumn('last_credit_pull_d',extractmonth(df_no_missing_cached['last_credit_pull_d']).cast(StringType())).\
withColumn('dti',df_no_missing_cached['dti'].cast(DoubleType())).\
withColumn('loan_amnt',df_no_missing_cached['loan_amnt'].cast(DoubleType())).\
withColumn('annual_inc',df_no_missing_cached['annual_inc'].cast(DoubleType()))

In [31]:
dfWithCrunch.select(['last_credit_pull_d']).printSchema()

root
 |-- last_credit_pull_d: string (nullable = true)



In [32]:
dfWithCrunch.select(['last_credit_pull_d']).distinct().show()

+------------------+
|last_credit_pull_d|
+------------------+
|               Oct|
|               Sep|
|               Dec|
|               Aug|
|               May|
|               Jun|
|               Feb|
|               Nov|
|               Mar|
|               Jan|
|               Apr|
|               Jul|
+------------------+



In [33]:
dfWithCrunch.count()

1429645

In [34]:
dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade']).show()

+---------+--------+-----+--------------+-----+
|loan_amnt|int_rate|  dti|home_ownership|grade|
+---------+--------+-----+--------------+-----+
|  25000.0|    9.16| 21.6|          RENT|    B|
|   9600.0|   10.75|18.98|          RENT|    B|
|   5000.0|    5.32| 1.59|           OWN|    A|
|   9000.0|   14.49|19.59|      MORTGAGE|    C|
|  21000.0|   11.99|10.33|      MORTGAGE|    C|
|  18400.0|   11.99|18.52|          RENT|    C|
|  26400.0|    9.16| 5.46|      MORTGAGE|    B|
|  16000.0|   12.99|27.15|      MORTGAGE|    C|
|  15075.0|    9.16|14.59|      MORTGAGE|    B|
|  25000.0|    7.39|11.47|           OWN|    A|
|  19600.0|   10.75|18.78|          RENT|    B|
|  20000.0|   13.67|11.48|           OWN|    C|
|  30000.0|   11.47|13.08|      MORTGAGE|    B|
|  40000.0|   22.45| 32.2|      MORTGAGE|    E|
|  30000.0|   11.99|14.38|      MORTGAGE|    C|
|  35000.0|    9.75|23.88|      MORTGAGE|    B|
|   3000.0|   19.48| 16.4|           OWN|    E|
|  12925.0|   18.99| 22.0|      MORTGAGE

In [35]:
dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- grade: string (nullable = true)



In [36]:
dfWithCrunch.select(['loan_amnt','annual_inc']).describe().show()

+-------+------------------+------------------+
|summary|         loan_amnt|        annual_inc|
+-------+------------------+------------------+
|  count|           1429645|           1429645|
|   mean|15366.699302973815| 81111.66344812172|
| stddev|  9641.85136599622|134222.54004920358|
|    min|            1000.0|               0.0|
|    max|           40000.0|             6.1E7|
+-------+------------------+------------------+



### Writing into a Hive Table for further Descriptive Analytics

In [37]:
dfWithCrunch.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [38]:
dfWithCrunch.write.format("parquet").mode('overwrite').parquet('/user/root/datacruncher/loan')

In [39]:
from pyspark import HiveContext

In [40]:
sqlStringADD = 'loan_amnt double,term string, int_rate string, installment double, grade string, emp_length string, home_ownership string, annual_inc double, verification_status string, loan_status string, purpose string, addr_state string, dti double, delinq_2yrs double, earliest_cr_line string, inq_last_6mths double, open_acc double, pub_rec double, revol_bal double, revol_util string, total_acc double, last_credit_pull_d string'

hc = HiveContext(sc)
hc.sql("DROP TABLE IF EXISTS personal_loan")
hc.sql("CREATE EXTERNAL TABLE personal_loan (" + sqlStringADD + """)
        STORED AS PARQUET
        LOCATION '/user/root/datacruncher/loan'""")

DataFrame[]

### Load from a Hive Table for Further efficient analytics.

In [41]:
rawhive_df = sqlContext.read.load("/user/root/datacruncher/loan").repartition(60).cache()
#rawhive_df.registerTempTable("crunched_data")

In [42]:
len(rawhive_df.columns)

22

In [43]:
rawhive_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [44]:
rawhive_df.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- grade: string (nullable = true)



In [45]:
rawhive_df.count()

1429645

### Data Transformation: Normalization of "annual_inc" and "loan_amnt"

In [46]:
from pyspark.sql.functions import *

max_annual_inc = rawhive_df.select(max('annual_inc')).collect()[0][0]

min_annual_inc = rawhive_df.select(min('annual_inc')).collect()[0][0]

#sqlContext.udf.register("t_annual_inc", lambda x: ((x-min_annual_inc)/(max_annual_inc-min_annual_inc)))

In [47]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [48]:
normalized_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

In [49]:
max_loan_amnt = rawhive_df.select(max('loan_amnt')).collect()[0][0]

min_loan_amnt = rawhive_df.select(min('loan_amnt')).collect()[0][0]

#sqlContext.udf.register("t_loan_amnt", lambda x: ((x-min_loan_amnt)/(max_loan_amnt-min_loan_amnt)))

In [50]:
def t_loan_amnt(origin):
    return ((origin-min_loan_amnt)/(max_loan_amnt-min_loan_amnt))

In [51]:
normalized_loan_amnt = udf(lambda x: t_loan_amnt(x),DoubleType())

In [52]:
rawhive_df.withColumn('loan_amnt',normalized_loan_amnt(rawhive_df['loan_amnt'])).select('loan_amnt').\
describe().show()

+-------+-------------------+
|summary|          loan_amnt|
+-------+-------------------+
|  count|            1429645|
|   mean| 0.3683769052044867|
| stddev|0.24722695810246717|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [53]:
rawhive_df.withColumn('annual_inc',normalized_annual_inc(rawhive_df['annual_inc'])).select('annual_inc').\
describe().show()

+-------+--------------------+
|summary|          annual_inc|
+-------+--------------------+
|  count|             1429645|
|   mean|0.001329699400788...|
| stddev| 0.00220036950900334|
|    min|                 0.0|
|    max|                 1.0|
+-------+--------------------+



In [54]:
normalized_df = rawhive_df.withColumn('loan_amnt',normalized_loan_amnt(rawhive_df['loan_amnt'])).\
withColumn('annual_inc',normalized_annual_inc(rawhive_df['annual_inc']))

In [55]:
normalized_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



### Number of data rows that are only "Fully Paid" and "Charged Off"

In [56]:
from pyspark.sql.functions import col

In [57]:
normalized_df.select(col('loan_status')).groupBy('loan_status').count().show()

+------------------+------+
|       loan_status| count|
+------------------+------+
|        Fully Paid|572976|
|           Default|    31|
|   In Grace Period|  7534|
|       Charged Off|152817|
|Late (31-120 days)| 18814|
|           Current|674110|
| Late (16-30 days)|  3363|
+------------------+------+



In [58]:
normalized_df.select(['loan_amnt','term','int_rate','installment','grade','annual_inc','revol_bal','loan_status']).\
orderBy(['loan_amnt','term','int_rate','installment','grade','annual_inc']).show()

+---------+----------+--------+-----------+-----+--------------------+---------+-----------+
|loan_amnt|      term|int_rate|installment|grade|          annual_inc|revol_bal|loan_status|
+---------+----------+--------+-----------+-----+--------------------+---------+-----------+
|      0.0| 36 months|    5.31|      30.12|    A|4.098360655737705E-4|     3215|    Current|
|      0.0| 36 months|    5.31|      30.12|    A|5.081967213114754E-4|    25993| Fully Paid|
|      0.0| 36 months|    5.31|      30.12|    A|5.901639344262295E-4|     3311| Fully Paid|
|      0.0| 36 months|    5.31|      30.12|    A|6.065573770491804E-4|     1605| Fully Paid|
|      0.0| 36 months|    5.31|      30.12|    A|6.557377049180328E-4|     1016| Fully Paid|
|      0.0| 36 months|    5.31|      30.12|    A|6.557377049180328E-4|     1329|    Current|
|      0.0| 36 months|    5.31|      30.12|    A|7.180327868852459E-4|      804|    Current|
|      0.0| 36 months|    5.31|      30.12|    A|9.344262295081968E-4|

In [59]:
normalized_df.select(['loan_amnt','annual_inc']).describe().show()

+-------+-------------------+--------------------+
|summary|          loan_amnt|          annual_inc|
+-------+-------------------+--------------------+
|  count|            1429645|             1429645|
|   mean|0.36837690520448685|0.001329699400788...|
| stddev|0.24722695810246717| 0.00220036950900334|
|    min|                0.0|                 0.0|
|    max|                1.0|                 1.0|
+-------+-------------------+--------------------+



In [60]:
normalized_filtered_df = normalized_df.filter(col('loan_status') == 'Fully Paid').\
union(normalized_df.filter(col('loan_status') == 'Charged Off'))

In [61]:
normalized_filtered_df.select(['loan_amnt']).describe().show()

+-------+-------------------+
|summary|          loan_amnt|
+-------+-------------------+
|  count|             725793|
|   mean| 0.3463430432785299|
| stddev|0.23682390199294334|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [62]:
data = normalized_filtered_df.repartition(60).cache()

In [63]:
data.select(['loan_amnt']).describe().show()

+-------+-------------------+
|summary|          loan_amnt|
+-------+-------------------+
|  count|             725793|
|   mean| 0.3463430432785285|
| stddev|0.23682390199294323|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



### Drop Null

In [64]:
data_no_missing_df = data.dropna(how='any')

In [65]:
data_no_missing_df.count()

725793

In [66]:
df_no_missing_cached.select(['loan_status']).distinct().show(100)

+------------------+
|       loan_status|
+------------------+
|        Fully Paid|
|           Default|
|   In Grace Period|
|       Charged Off|
|Late (31-120 days)|
|           Current|
| Late (16-30 days)|
+------------------+



In [67]:
data_no_missing_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



# 3. Data Modeling

In [68]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, VectorIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [69]:
labelIndexer = StringIndexer(inputCol='loan_status',outputCol='indexedLabel')

In [70]:
homeIndexer = StringIndexer(inputCol='home_ownership',outputCol='homeIndexed')
homeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='homeIndexed',outputCol='homeVec')

In [71]:
gradeIndexer = StringIndexer(inputCol='grade',outputCol='gradeIndexed')
gradeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='gradeIndexed',outputCol='gradeVec')

In [72]:
featureAssembler = VectorAssembler(inputCols=['loan_amnt','int_rate','dti','homeVec','gradeVec'],\
                                   outputCol='***features')

In [73]:
dt = DecisionTreeClassifier(featuresCol='***features',labelCol='indexedLabel')

In [74]:
pipeline_dt = Pipeline().setStages([gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,labelIndexer,featureAssembler,dt])

In [75]:
data_no_missing_df.select(['loan_amnt']).describe().show()

+-------+-------------------+
|summary|          loan_amnt|
+-------+-------------------+
|  count|             725793|
|   mean|0.34634304327852844|
| stddev|0.23682390199294323|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [76]:
training , test = data_no_missing_df.randomSplit([0.6,0.4])

In [77]:
model1 = pipeline_dt.fit(training)

In [78]:
training.select(['loan_amnt','int_rate','dti','home_ownership','grade']).describe().show()

+-------+-------------------+------------------+------------------+--------------+------+
|summary|          loan_amnt|          int_rate|               dti|home_ownership| grade|
+-------+-------------------+------------------+------------------+--------------+------+
|  count|             434960|            434960|            434960|        434960|434960|
|   mean| 0.3461923421782374|13.215111067685006|18.710918406290233|          null|  null|
| stddev|0.23685318932060942| 5.158257266556698|13.392276063257604|          null|  null|
|    min|                0.0|              5.31|              -1.0|           ANY|     A|
|    max|                1.0|             30.99|             999.0|          RENT|     G|
+-------+-------------------+------------------+------------------+--------------+------+



In [79]:
training.select(['loan_amnt','int_rate','dti','home_ownership','grade']).show()

+---------+--------+-----+--------------+-----+
|loan_amnt|int_rate|  dti|home_ownership|grade|
+---------+--------+-----+--------------+-----+
|      0.0|    6.97|12.47|      MORTGAGE|    A|
|      0.0|    6.99| 5.52|          RENT|    A|
|      0.0|    7.02|10.64|          RENT|    A|
|      0.0|    7.21|19.47|          RENT|    A|
|      0.0|    7.39|  4.8|          RENT|    A|
|      0.0|    7.97| 8.42|      MORTGAGE|    A|
|      0.0|    7.99|16.84|      MORTGAGE|    A|
|      0.0|    8.24|10.85|      MORTGAGE|    B|
|      0.0|    8.24| 2.46|          RENT|    B|
|      0.0|    9.43| 11.5|          RENT|    B|
|      0.0|    9.49|27.53|      MORTGAGE|    B|
|      0.0|   10.08|14.88|          RENT|    B|
|      0.0|   10.49|13.24|      MORTGAGE|    B|
|      0.0|   11.39|20.65|          RENT|    B|
|      0.0|   11.49|26.24|          RENT|    B|
|      0.0|   11.49| 8.58|          RENT|    B|
|      0.0|   11.49|12.66|           OWN|    B|
|      0.0|   11.98|14.72|          RENT

# 4. Evaluation of Models

In [80]:
result = model1.transform(test)

In [81]:
test.count()

290833

In [82]:
training.count()

434960

In [83]:
training.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|343652|
|Charged Off| 91308|
+-----------+------+



In [84]:
training.filter(training.loan_status == 'Fully Paid').count()

343652

In [85]:
training.filter(training.loan_status == 'Charged Off').count()

91308

In [86]:
result.select('loan_status','indexedLabel','prediction').filter(result.loan_status != 'Fully Paid').show(10)

+-----------+------------+----------+
|loan_status|indexedLabel|prediction|
+-----------+------------+----------+
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
|Charged Off|         1.0|       0.0|
+-----------+------------+----------+
only showing top 10 rows



In [87]:
result.select('loan_status','indexedLabel','prediction').filter(result.loan_status != 'Fully Paid').count()

61509

In [88]:
result.select('loan_status','indexedLabel','prediction').filter(result.loan_status != 'Fully Paid').\
filter(result.indexedLabel == result.prediction).count()

4273

In [89]:
result.select('loan_status','indexedLabel','prediction').filter(result.loan_status != 'Fully Paid').\
filter(result.indexedLabel != result.prediction).count()

57236

In [90]:
result.select('loan_status','indexedLabel','prediction').filter(result.loan_status == 'Fully Paid').show(20)

+-----------+------------+----------+
|loan_status|indexedLabel|prediction|
+-----------+------------+----------+
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
| Fully Paid|         0.0|       0.0|
+-----------+------------+----------+
only showing top 20 rows



In [91]:
result.select('loan_status','indexedLabel','prediction').filter(result.indexedLabel != result.prediction).\
filter(result.loan_status == 'Fully Paid').count()

3994

In [92]:
result.select('loan_status','indexedLabel','prediction').filter(result.indexedLabel != result.prediction).filter(result.loan_status == 'Charged Off').count()

57236

## Calculation for Confusion Matrix.

In [93]:
def eval_metrics(lap):

    tp = float(len(lap[(lap['indexedLabel']==0) & (lap['prediction']==0)]))

    tn = float(len(lap[(lap['indexedLabel']==1) & (lap['prediction']==1)]))

    fp = float(len(lap[(lap['indexedLabel']==1) & (lap['prediction']==0)]))

    fn = float(len(lap[(lap['indexedLabel']==0) & (lap['prediction']==1)]))

    positivepredictivevalue = tp / (tp+fp)
    negativepredictivevalue = tn / (tn+fn)

    sensitivity = tp / (tp+fn)
    specificity = tn / (tn+fp)

    accuracy = (tp+tn) / (tp+tn+fp+fn)

    return {'PPV': positivepredictivevalue, 'NPV':negativepredictivevalue,'Sensitivity': sensitivity, 'Specificity': specificity, 'Accuracy': accuracy}

In [94]:
! pip install pandas

In [95]:
lap = result.select("indexedLabel", "prediction").toPandas()

In [96]:
m = eval_metrics(lap)

In [97]:
m

{'PPV': 0.7974420135472774,
 'NPV': 0.5168743195838877,
 'Sensitivity': 0.9825835935183409,
 'Specificity': 0.0694695085272074,
 'Accuracy': 0.7894668074118136}

In [98]:
tp = float(len(lap[(lap['indexedLabel']==0) & (lap['prediction']==0)])) #tp - Full paid

In [99]:
tn = float(len(lap[(lap['indexedLabel']==1) & (lap['prediction']==1)])) #tn - Charged off

In [100]:
fp = float(len(lap[(lap['indexedLabel']==1) & (lap['prediction']==0)])) #fp

In [101]:
fn = float(len(lap[(lap['indexedLabel']==0) & (lap['prediction']==1)])) #fn

In [102]:
! pip install plotly

In [103]:
! pip install chart-studio

In [104]:
import chart_studio.plotly as py
py.sign_in('grizzly', 'MHH5wgaO3T1lxbUFgeT5')
from plotly.graph_objs import *
import plotly.graph_objs as go
import pandas as pd
import requests
requests.packages.urllib3.disable_warnings()

In [105]:
! ls images/confusion_matrix.png

ls: cannot access 'images/confusion_matrix.png': No such file or directory


In [106]:
![title](images/confusion_matrix.png)

/bin/sh: 1: Syntax error: word unexpected (expecting ")")


In [107]:
labelIndexer.fit(training).transform(training).\
select(['loan_status','indexedLabel']).groupBy(['loan_status','indexedLabel']).count().show()

+-----------+------------+------+
|loan_status|indexedLabel| count|
+-----------+------------+------+
| Fully Paid|         0.0|343652|
|Charged Off|         1.0| 91308|
+-----------+------------+------+



In [108]:
labelIndexer.fit(test).transform(test).\
select(['loan_status','indexedLabel']).groupBy(['loan_status','indexedLabel']).count().show()

+-----------+------------+------+
|loan_status|indexedLabel| count|
+-----------+------------+------+
| Fully Paid|         0.0|229324|
|Charged Off|         1.0| 61509|
+-----------+------------+------+



## 0: Positive: Fully Paid, 1: Negative: Charged Off

In [109]:
! hdfs dfs -ls /user/root/datacruncher

Found 1 items
drwxr-xr-x   - root hadoop          0 2019-10-11 15:39 /user/root/datacruncher/loan


In [110]:
training.count()

434960

In [111]:
test.count()

290833

In [112]:
result.select(["indexedLabel", "prediction"]).show()

+------------+----------+
|indexedLabel|prediction|
+------------+----------+
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         1.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
|         0.0|       0.0|
+------------+----------+
only showing top 20 rows



In [113]:
tp, fp

(225330.0, 57236.0)

In [114]:
tn, fn

(4273.0, 3994.0)

In [115]:
! pip install sklearn

In [116]:
! pip install scipy

In [117]:
f_status = udf(lambda x: status_name(x), StringType())

In [120]:
#model1.write().overwrite().save('gs://grizzly-gkbm/loanpayment/')